In [ ]:
import sympy as sp
import numpy as np
from scipy.optimize import fsolve
from numpy.linalg import LinAlgError

## 必要なパラメータの定義

In [ ]:
# ヘノン写像に関するパラメータの設定
a_val = 1.4
b_val = 0.3
n = 10

# ニュートン法の初期値の設定
initial_guess = [-10, -10]

# ニュートン法に関するパラメータの設定(defaultと同じ)
tol=1e-8
max_iter=100

# ランダムに生成する初期値に関するパラメータの設定
seed = 42
num_points = 10

## $n$ 回合成したヘノン写像を生成する

In [ ]:
# シンボリック変数の定義
x, y = sp.symbols('x y')
a, b = sp.symbols('a b')

# エノン写像の定義
def henon_map(x, y, a, b):
    return a - x**2 + b*y, x

# エノン写像を n 回合成する関数
def henon_n_compose(x, y, a, b, n, expand=False):
    xn, yn = x, y
    for i in range(n):
        xn, yn = henon_map(xn, yn, a, b)

    if expand:
        xn = sp.expand(xn)
        yn = sp.expand(yn)
    return xn, yn

In [ ]:
# n 回合成したエノン写像
xn, yn = henon_n_compose(x, y, a, b, n)
# print(f"H^{n}(x, y):")
# print(f"x_{n} =", xn)
# print(f"y_{n} =", yn)

## ニュートン法を用いて $n$ 回合成したヘノン写像の不動点を計算する

In [ ]:
# 指定された範囲からランダムな初期値を生成する関数（シード固定）
def generate_random_initial_guesses(num_points, lower_bound=-1, upper_bound=1, seed=None):
    if seed is not None:
        np.random.seed(seed)
    initial_guesses = []
    for _ in range(num_points):
        x0 = round(np.random.uniform(lower_bound, upper_bound), 1)
        y0 = round(np.random.uniform(lower_bound, upper_bound), 1)
        initial_guesses.append((x0, y0))
    return initial_guesses

initial_guesses = generate_random_initial_guesses(num_points, seed=seed)
print(initial_guesses)

In [ ]:
# シンボリック変数の再定義
x0, y0 = sp.symbols('x0 y0')

# エノン写像の合成結果を元に関数定義
xn, yn = henon_n_compose(x0, y0, a, b, n)

# 関数の定義
f1 = xn - x0
f2 = yn - y0
F = sp.Matrix([f1, f2])

# ヤコビ行列の計算
J = F.jacobian([x0, y0])

# 数値関数への変換
F_func = sp.lambdify((x0, y0, a, b), F, 'numpy')
J_func = sp.lambdify((x0, y0, a, b), J, 'numpy')

# ニュートン法の実装
def newton_method(F_func, J_func, initial_guess, a_val, b_val, tol=1e-8, max_iter=100):
    xy = np.array(initial_guess, dtype=float)
    for _ in range(max_iter):
        F_val = np.array(F_func(xy[0], xy[1], a_val, b_val), dtype=float).flatten()
        J_val = np.array(J_func(xy[0], xy[1], a_val, b_val), dtype=float)
        try:
            delta = np.linalg.solve(J_val, -F_val)
        except LinAlgError:
            delta = np.linalg.pinv(J_val).dot(-F_val)  # 擬似逆行列を使用
        xy += delta
        if np.linalg.norm(delta) < tol:
            break
    return xy

# ニュートン法による周期点の計算
periodic_point = newton_method(F_func, J_func, initial_guess, a_val, b_val)

print("周期点:", periodic_point)

## 実験データ収集用

In [ ]:
solutions = []

for initial_guess in initial_guesses:
    periodic_point = newton_method(F_func, J_func, initial_guess, a_val, b_val)
    print(f"{n}周期点:", f"初期値:{initial_guess}", periodic_point)
    solutions.append([initial_guess, periodic_point])

## [確認用] 周期点が初期値に近い値になっているかを計算

In [ ]:
# 検証用の関数
def verify_solution(solution, a, b, n):
    x, y = solution
    for _ in range(n):
        x, y = henon_map(x, y, a, b)
    return (x, y)

In [ ]:
verification_results = []
for initial, fixed in solutions:
    if not np.isnan(fixed).any():
        verified = verify_solution(fixed, a_val, b_val, n)
        verification_results.append((fixed, verified))

# 検証結果を表示
print("\nVerification Results:")
for fixed, verified in verification_results:
    print(f"Fixed point: {fixed} -> After {n} compositions: {verified}")

## [確認用] $n=1$の不動点を計算する

In [ ]:
def find_fixed_points(a_val, b_val):
    # シンボリック変数の定義
    x, y = sp.symbols('x y')

    # エノン写像の定義
    henon_map = (a_val - x**2 + b_val*y, x)

    # 不動点の条件
    fixed_point_eq1 = sp.Eq(henon_map[0], x)
    fixed_point_eq2 = sp.Eq(henon_map[1], y)

    # 連立方程式を解く
    solutions = sp.solve([fixed_point_eq1, fixed_point_eq2], (x, y))

    return solutions

# パラメータの設定
a_val = 1.4
b_val = 0.3

# 不動点を計算
fixed_points = find_fixed_points(a_val, b_val)
print(f"不動点: {fixed_points}")